## Importação de bibliotecas

In [ ]:
import pandas as pd
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('ticks')

## Leitura do Dataset

In [ ]:
colunas = ['regiao','uf','secao','subclasse','saldomovimentacao','cbo2002ocupacao','categoria','graudeinstrucao','idade','racacor','sexo','tipomovimentacao','tipodedeficiencia','salario','indicadoraprendiz']
df = pd.read_csv(r'C:\Users\Pichau\Documents\Fatec\3Semestre\Estrutura_de_dados\CAGED\CAGEDLIGTH\CAGEDLIGTH.csv', encoding = 'UTF-8', delimiter=";", usecols= colunas)

In [ ]:
#caso o data frame seja lido em iso-8859-1
'''
df = df.rename(columns = {'competÃªncia': 'competência', 
'regiÃ£o': 'região',
'municÃ­pio': 'município',
'seÃ§Ã£o': 'seção',
'saldomovimentaÃ§Ã£o': 'saldomovimentação',
'cbo2002ocupaÃ§Ã£o' : 'cbo2002ocupação',
'graudeinstruÃ§Ã£o': 'graudeinstrução',
'raÃ§acor': 'raçacor',
'tipomovimentaÃ§Ã£o': 'tipomovimentação',
'tipodedeficiÃªncia': 'tipodedeficiência',
'salÃ¡rio': 'salário'}, inplace = False)
'''

In [ ]:
df.info()

In [ ]:
df['sexo']

## Data cleaning

In [ ]:
df.isnull().sum()

### Como poucos valores estão nulos, vamos excluir estes registros, pois acreditos que não vai influenciar na análise

In [ ]:
df = df.dropna()

In [ ]:
df['salario']=df['salario'].apply(lambda x :  str(x).replace(',','.'))
df['salario']=df['salario'].apply(lambda x :  str(x).replace('nan','0'))
df['salario']=pd.to_numeric(df['salario'], downcast="float")

In [ ]:
df['admitidos/desligados'] = df['tipomovimentacao'].apply(lambda x: "Admitido" if x==10 or x==20 or x==25 or x==35 or x==70 else "Desligado")

In [ ]:
Raca = {1: 'BRANCA',2: 'PRETA',3: 'PARDA', 4: 'AMARELA',5 : 'INDIGENA',6:'NÃO INFORMADA',  9: 'NAO IDENTIFICADO'}
Sexo = {1: 'MASCULINO',3 :'FEMININO', 9: 'NAO IDENTIFICADO'}
AdmDem = {1: 'ADMISSÃO', 2: 'DESLIGAMENTO'}
TipoMovDesagregado = {
10:'Admissão por Primeiro Emprego',
20:'Admissão por Reemprego', 
25:'Admissão por contrato trabalho prazo determinado',
31:'Desligamento por Demissão sem Justa Causa', 
32:'Desligamento por Demissão com Justa Causa', 
33:'Culpa Recíproca',
35:'Admissão por Reintegraçao', 
40:'Desligamento a Pedido',
43:'Término contrato trabalho prazo determinado',
45:'Desligamento por Término de Contrato',
50:'Desligamento por Aposentadoria', 
60:'Desligamento por morte',
70:'Admissão por Transferência', 
80:'Desligamento por Transferência', 
90:'Desliamento por Acordo Empregado e Empregador',
98:'Desligamento de Tipo Ignorado',
99:'Não Identificado'}

In [ ]:
df['racacor'] = df['racacor'].map(Raca)
df['sexo'] = df['sexo'].map(Sexo)
df['tipomovimentacao'] = df['tipomovimentacao'].map(TipoMovDesagregado)

In [ ]:
def categorizarClasseSocial(x):
    #try:
        #x=float(x)

        if x<2200:
            return 'E'
        elif x<4400:
            return 'D'
        elif x<11000:
            return 'C'
        elif x<22000:
            return 'B'
        else:
            return 'A'

#    except:
 #       print("caca: "+x)

In [ ]:
df['classe social'] = df['salario'].apply(lambda x: categorizarClasseSocial(x))

In [ ]:
sns.catplot(x="classe social", kind="count", data=df)

In [ ]:
def categorizarFaixaEtaria(x):
        if x<20:
            return "Adolescência"
        elif x<26:
            return "Pós-adolescência"
        elif x<40:
            return "Adultidade"
        elif x<65:
            return "Meia-idade"
        elif x<80:
            return "Terceira idade"
        else:
            return "Quarta Idade"

In [ ]:
df['faixa Etária'] = df['idade'].apply(lambda x: categorizarFaixaEtaria(x))

### Criar coluna admitidos/desligados apartir dos dados de tipomovimentação

### Calcular total admitido e total desligado

In [ ]:
total = df['admitidos/desligados'].count()

### Cria um array só com admitidos

In [ ]:
totalAdmitidoArray = df[df['admitidos/desligados'] == "Admitido"]
totalAdmitido = totalAdmitidoArray['admitidos/desligados'].count()

### Cria um array só com desligados

In [ ]:
totalDesligadoArray = df[df['admitidos/desligados'] == "Desligado"]
totalDesligado = total - totalAdmitido

# Tópico 1 - Qual foi o índice de pessoas empregadas e desempregadas em certo período, categorizadas por:
    - Gênero 
    - Classe social 
    - Faixa Etária 
    - Nível de escolaridade 
    - Setores (Agronegócio, Indústria, Comércio, etc...) 

# Começa a calcular os items pedidos

## Indice sexo

### Admitidos

In [ ]:
TotalAdmitidoFem = (totalAdmitidoArray[totalAdmitidoArray['sexo']=='FEMININO'])['admitidos/desligados'].count()
TotalAdmitidoMasc = totalAdmitido - TotalAdmitidoFem
porcentagemAdmitidoFem = ((TotalAdmitidoFem/totalAdmitido)*100).astype(int)
porcentagemAdmitidoMasc = 100-porcentagemAdmitidoFem
f'Porcentagem Admitido feminino: {porcentagemAdmitidoFem}%  ||  Porcentagem Admitido Masculino:  {porcentagemAdmitidoMasc}%'

In [ ]:
#sns.catplot(x="sexo", y="TotalAdmitidoFemi",  palette="ch:.25",kind="bar", data=df)

### Desligados

In [ ]:
TotalDesligadoFem = (totalDesligadoArray[totalDesligadoArray['sexo']=='FEMININO'])['admitidos/desligados'].count()
TotalDesligadoMasc = totalDesligado - TotalDesligadoFem
porcentagemDesligadoFem = ((TotalDesligadoFem/totalDesligado)*100).astype(int)
porcentagemDesligadoMasc = 100-porcentagemDesligadoFem
f'Porcentagem Desligado feminino: {porcentagemDesligadoFem}%  ||  Porcentagem Desligado Masculino: {porcentagemDesligadoMasc}%'

## Indice Classe social

In [ ]:
classes=['A','B','C','D','E']

### Admitidos

In [ ]:
for x in classes: 
    totalAdmitidoClasse = (totalAdmitidoArray[totalAdmitidoArray['classe social']==x])['admitidos/desligados'].count()
    porcentagemAdmitido = ((totalAdmitidoClasse/totalAdmitido)*100).astype(int)
    print(f'Porcentagem Admitidos classe {x}: {porcentagemAdmitido}%')

### Desligados

In [ ]:
for x in classes: 
    totalDesligadoClasse = (totalDesligadoArray[totalDesligadoArray['classe social']==x])['admitidos/desligados'].count()
    porcentagemDesligado = ((totalDesligadoClasse/totalDesligado)*100).astype(int)
    print(f'Porcentagem Desligado classe {x}: {porcentagemDesligado}%')

## Indice faixa etaria

### Classificação faixa etaria
    15 a 20 anos: Adolescência
    20 a 26 anos: Pós-adolescência
    26 a 40 anos: Adultidade
    40 a 65 anos: Meia-idade
    65 a 80 anos: Terceira idade
    mais que 80 anos: Quarta Idade

In [ ]:
faixaEtarias = {
    "Adolescência":"15 a 20 anos",
    "Pós-adolescência":"20 a 26 anos",
    "Adultidade":"26 a 40 anos",
    "Meia-idade":"40 a 65 anos",
    "Terceira idade":"65 a 80 anos",
    "Quarta Idade":"mais que 80 anos"
    }

### Admitidos

In [ ]:
for x,y in faixaEtarias.items():
    totalAdmitidoFaixaEtaria = (totalAdmitidoArray[totalAdmitidoArray['faixa Etária']==x])['admitidos/desligados'].count()
    porcentagemAdmitido = ((totalAdmitidoFaixaEtaria/totalAdmitido)*100).astype(int)
    print(f'Porcentagem Admitidos {x} ({y}): {porcentagemAdmitido}%')

In [ ]:
sns.catplot(x="admitidos/desligados",hue="faixa Etária", kind="count", palette="ch:.25", data=df)

In [ ]:
sns.catplot(x="admitidos/desligados", y="salario", hue="faixa Etária", kind="bar", data=df)

### Desligados

In [ ]:
for x,y in faixaEtarias.items():
    totalDesligadoFaixaEtaria = (totalDesligadoArray[totalDesligadoArray['faixa Etária']==x])['admitidos/desligados'].count()
    porcentagemDesligado = ((totalDesligadoFaixaEtaria/totalDesligado)*100).astype(int)
    print(f'Porcentagem Desligado {x} ({y}): {porcentagemDesligado}%')

## Indice Grau Instrução

In [ ]:
grauInstrução = {
    1:"Analfabeto",
    2:"Até 5ª Incompleto",
    3:"5ª Completo Fundamental",
    4:"6ª a 9ª Fundamental",
    5:"Fundamental Completo",
    6:"Médio Incompleto",
    7:"Médio Completo",
    8:"Superior Incompleto",
    9:"Superior Completo",
    10:"Mestrado",
    11:"Doutorado",
    80:"Pós-Graduação completa",
    99:"Não Identificado"
}

### Admitidos

In [ ]:
for x,y in grauInstrução.items():
    totalAdmitidoGrauInstrução = (totalAdmitidoArray[totalAdmitidoArray['graudeinstrucao']==x])['admitidos/desligados'].count()
    porcentagemAdmitido = ((totalAdmitidoGrauInstrução/totalAdmitido)*100).astype(int)
    print(f'Porcentagem Admitidos {y}: {porcentagemAdmitido}%')

### Desligados

In [ ]:
for x,y in grauInstrução.items():
    totalDesligadoGrauInstrução = (totalDesligadoArray[totalDesligadoArray['graudeinstrucao']==x])['admitidos/desligados'].count()
    porcentagemDesligado = ((totalDesligadoGrauInstrução/totalDesligado)*100).astype(int)
    print(f'Porcentagem Desligado {y}: {porcentagemDesligado}%')

In [ ]:
df['graudeinstrucao'] = df['graudeinstrucao'].map(grauInstrução)

In [ ]:
sns.catplot(x="admitidos/desligados",hue="graudeinstrucao", kind="count", data=df)

In [ ]:
sns.catplot(x="admitidos/desligados", y="salario", hue="graudeinstrucao", kind="bar", data=df)

## Indice Setores

In [ ]:
setores ={
    "A":"Agricultura, Pecuária, Produção Florestal, Pesca e AqÜIcultura",
    "B":"Indústrias Extrativas",
    "C":"Indústrias de Transformação",
    "D":"Eletricidade e Gás",
    "E":"Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação",
    "F":"Construção",
    "G":"Comércio, Reparação de Veículos Automotores e Motocicletas",
    "H":"Transporte, Armazenagem e Correio",
    "I":"Alojamento e Alimentação",
    "J":"Informação e Comunicação",
    "K":"Atividades Financeiras, de Seguros e Serviços Relacionados",
    "L":"Atividades Imobiliárias",
    "M":"Atividades Profissionais, Científicas e Técnicas",
    "N":"Atividades Administrativas e Serviços Complementares",
    "O":"Administração Pública, Defesa e Seguridade Social",
    "P":"Educação",
    "Q":"Saúde Humana e Serviços Sociais",
    "R":"Artes, Cultura, Esporte e Recreação",
    "S":"Outras Atividades de Serviços",
    "T":"Serviços Domésticos",
    "U":"Organismos Internacionais e Outras Instituições Extraterritoriais",
    "Z":"Não identificado"
}

In [ ]:
df['secao'].sample(10)

### Admitidos

In [ ]:
df['secao']= df['secao'].map(setores)

In [ ]:
ax = plt.subplots(figsize=(7, 7))
sns.countplot(y="secao", data=df, palette="ch:.25")

In [ ]:
for x,y in setores.items():
    totalAdmitidoSetores = (totalAdmitidoArray[totalAdmitidoArray['secao']==x])['admitidos/desligados'].count()
    porcentagemAdmitido = ((totalAdmitidoSetores/totalAdmitido)*100).astype(int)
    print(f'Porcentagem Admitidos do setor: {y}: {porcentagemAdmitido}%')

### Desligados

In [ ]:
for x,y in setores.items():
    totalDesligadoSetores = (totalDesligadoArray[totalDesligadoArray['secao']==x])['admitidos/desligados'].count()
    porcentagemDesligado = ((totalDesligadoSetores/totalDesligado)*100).astype(int)
    print(f'Porcentagem Desligado do setor: {y}: {porcentagemDesligado}%')

# Tópico 2

# Tópico 3 

In [ ]:
## Quantidade de admissões/demissões por gênero. 


In [ ]:
SEXO ={
    "MASCULINO":"MASCULINO",
    "FEMININO":"FEMININO",
    "NAO IDENTIFICADO":"NÃO ESPECIFICADO"}

In [ ]:
df['sexo']= df['sexo'].map(SEXO)

In [ ]:
for x,y in SEXO.items():
    totalAdmitidosexo = (totalAdmitidoArray[totalAdmitidoArray['sexo']==x])['admitidos/desligados'].count()
    porcentagemAdmitido = ((totalAdmitidosexo/totalAdmitido)*100).astype(int)
    print(f'Porcentagem Admitidos pelo sexo: {y}: {porcentagemAdmitido}%')

In [ ]:
for x,y in SEXO.items():
    totalDesligadosexo = (totalDesligadoArray[totalDesligadoArray['sexo']==x])['admitidos/desligados'].count()
    porcentagemDesligado = ((totalDesligadosexo/totalDesligado)*100).astype(int)
    print(f'Porcentagem Desligado pelo sexo: {y}: {porcentagemDesligado}%')

In [ ]:
ax = plt.subplots(figsize=(9, 7))
sns.countplot(y="admitidos/desligados", hue="sexo", data=df, palette="ch:.25")

# Tópico 4 

In [ ]:
df.groupby(by='sexo')['idade'].mean()

In [ ]:
df.groupby(by='sexo')['salario'].mean()

In [ ]:
df.groupby(by='sexo')['salario'].max()

In [ ]:
df['sexo'].value_counts()

In [ ]:
fig = plt.figure(figsize=(10, 5))
male_only = df[df['sexo'] == 'MASCULINO']
female_only = df[df['sexo'] == 'FEMININO']


fig = plt.figure(figsize=(5, 5))

colors = ['#AFAFF5', '#EFAFB5']
labels = ['Masculino', 'Feminino']
sizes = [len(male_only), len(female_only)]
explode = (0.05, 0)
sns.set(font_scale=1.5)
ax = plt.pie(sizes, labels=labels, explode=explode, colors=colors, shadow=True, startangle=90, autopct='%1.f%%')

plt.title('% Percentual de gênero no dataset')
plt.show()

## Média salarial por raça/cor

In [ ]:
df.groupby(by='racacor')['salario'].mean()

In [ ]:
sns.catplot(x="sexo", y="salario", hue="racacor", kind="bar", data=df)

In [ ]:
sns.catplot(x="admitidos/desligados", kind="count", palette="ch:.25", data=df)

In [ ]:
df['tipomovimentacao'].value_counts()

In [ ]:
df[df['admitidos/desligados'] == 'Desligado']['tipomovimentacao'].value_counts()

In [ ]:
ax = plt.subplots(figsize=(7, 7))
sns.countplot(y="tipomovimentacao", data=df, palette="ch:.25")

In [ ]:
sns.catplot(x="sexo", y="salario",  palette="ch:.25",kind="bar", data=df)

In [ ]:
df.groupby(by=['sexo','admitidos/desligados'])['salario'].mean()

In [ ]:
sns.catplot(y="admitidos/desligados", hue="sexo", kind="count",
            palette="ch:.25", edgecolor=".3",
            data=df)

##  Diferença salarial entre homens e mulheres admitidos e demitidos

In [ ]:
sns.catplot(x="sexo",palette="ch:.25", y="salario", hue="admitidos/desligados", kind="bar", data=df)

In [1]:
# Tópico 5 

In [103]:
df.groupby(by='sexo')['tipomovimentacao'].value_counts()

sexo       tipomovimentacao                                
FEMININO   Admissão por Reemprego                              500069
           Desligamento por Demissão sem Justa Causa           328517
           Desligamento a Pedido                               145644
           Término contrato trabalho prazo determinado          91664
           Admissão por Primeiro Emprego                        39625
           Desliamento por Acordo Empregado e Empregador         5884
           Desligamento por Demissão com Justa Causa             5683
           Desligamento por Término de Contrato                  5254
           Não Identificado                                      2293
           Admissão por contrato trabalho prazo determinado      2121
           Desligamento por morte                                1556
           Admissão por Reintegraçao                             1065
           Culpa Recíproca                                        860
           Desligamento por Ap

In [104]:
MOTIVO= {
'Admissão por Primeiro Emprego':'Admissão por Primeiro Emprego',
'Admissão por Reemprego':'Admissão por Reemprego', 
'Admissão por contrato trabalho prazo determinado':'Admissão por contrato trabalho prazo determinado',
'Desligamento por Demissão sem Justa Causa':'Desligamento por Demissão sem Justa Causa', 
'Desligamento por Demissão com Justa Causa':'Desligamento por Demissão com Justa Causa', 
'Culpa Recíproca':'Culpa Recíproca',
'Admissão por Reintegraçao':'Admissão por Reintegraçao', 
'Desligamento a Pedido':'Desligamento a Pedido',
'Término contrato trabalho prazo determinado':'Término contrato trabalho prazo determinado',
'Desligamento por Término de Contrato':'Desligamento por Término de Contrato',
'Desligamento por Aposentadoria':'Desligamento por Aposentadoria', 
'Desligamento por morte':'Desligamento por morte',
'Admissão por Transferência':'Admissão por Transferência', 
'Desligamento por Transferência':'Desligamento por Transferência', 
'Desliamento por Acordo Empregado e Empregador':'Desliamento por Acordo Empregado e Empregador',
'Desligamento de Tipo Ignorado':'Desligamento de Tipo Ignorado',
'Não Identificado':'Não Identificado'}

In [107]:
df['tipomovimentacao']= df['tipomovimentacao'].map(MOTIVO)

In [122]:
totalmotivosArray = df['admitidos/desligados']
totalmotivoscount = df['admitidos/desligados'].count()

In [124]:
for x,y in MOTIVO.items():
    totalmotivo = (totalmotivosArray[totalmotivosArray['sexo']=='FEMININO'])['tipomovimentacao'].count()
    porcentagemmotivo = ((totalmotivo/total)*100).astype(int)
    print(f'Porcentagem dos motivos por gênero: {y}: {porcentagemmotivo}%')

KeyError: 'sexo'

In [ ]:
fig = plt.figure(figsize=(10, 5))
male_only = df[df['sexo'] == 'MASCULINO']
female_only = df[df['sexo'] == 'FEMININO']


fig = plt.figure(figsize=(5, 5))

colors = ['#AFAFF5', '#EFAFB5']
labels = ['Masculino', 'Feminino']
sizes = [len(male_only), len(female_only)]
explode = (0.05, 0)
sns.set(font_scale=1.5)
ax = plt.pie(sizes, labels=labels, explode=explode, colors=colors, shadow=True, startangle=90, autopct='%1.f%%')

plt.title('% Percentual de gênero no dataset')
plt.show()